In [8]:
import dspy
import json
import langextract as lx
from pathlib import Path
from dotenv import load_dotenv
from rich.console import Console

load_dotenv()

cons = Console()

In [2]:
path = Path("../data/radgraphXL/cleaned_data.jsonl")
assert path.exists()

In [11]:
with open(path, 'r') as f:
    data = [json.loads(line) for line in f]
print(f"Loaded {len(data)} reports from RadGraph-XL dataset.")
print(data[0].keys())
print(data[0]['ner'])

Loaded 2300 reports from RadGraph-XL dataset.
dict_keys(['dataset', 'doc_key', 'tokens', 'ner', 'relations'])
[[77, 77, 'Anatomy::definitely present'], [78, 78, 'Observation::definitely present'], [83, 83, 'Anatomy::definitely present'], [84, 84, 'Anatomy::definitely present'], [86, 86, 'Observation::definitely present'], [89, 89, 'Anatomy::definitely present'], [90, 90, 'Observation::definitely absent'], [92, 96, 'Anatomy::definitely present'], [97, 98, 'Observation::definitely present'], [102, 107, 'Observation::definitely present'], [112, 112, 'Anatomy::definitely present'], [114, 114, 'Anatomy::definitely present'], [115, 115, 'Observation::definitely absent'], [117, 119, 'Observation::definitely present'], [126, 128, 'Anatomy::definitely present'], [133, 133, 'Anatomy::definitely present'], [134, 134, 'Observation::definitely absent'], [137, 137, 'Observation::definitely absent'], [139, 139, 'Anatomy::definitely present'], [140, 140, 'Observation::definitely absent'], [142, 142, '

In [9]:
sample = data[0]
rad_text = " ".join(sample['tokens'])
cons.print(rad_text)

STUDY : CT torso . HISTORY : Metastatic breast cancer and recently diagnosed squamous cell cancer . Restaging . 
COMPARISONS : _ _ _ . TECHNIQUE : Following the uneventful administration of intravenous contrast , MDCT images 
were acquired from the thoracic inlet to the pubic symphysis and displayed in axial , coronal and sagittal 
reconstructions . Three - minute delayed images through the abdomen were also acquired . CT OF THE CHEST WITH IV 
CONTRAST : Left mastectomy is again evident . Heart size is normal . No pericardial effusion . Right upper 
paratracheal and prevascular lymph nodes measure up to 5 mm , within normal range . There is no axillary or hilar 
adenopathy . A few subpleural blebs are again evident at the right lung apex . There are no pulmonary nodules . No 
consolidation or pleural effusion . Bibasilar dependent atelectasis . CT OF THE ABDOMEN WITH IV CONTRAST : There 
are numerous low - attenuation lesions throughout the liver , many of which are new compared to the CT of _ _ _ . 
The lesions seen previously have increased in size in the interim . For example , lesion at the dome of the liver 
measures 1 . 5 cm , previously 6 mm ( 7 : 31 ) . A 2 . 4 x 2 . 5 cm lesion within segment VIII was not definitely 
present previously ( 7 : 35 ) . A lesion located within segment V measures 3 . 2 x 2 . 3 cm , previously 2 . 1 x 1 
. 5 cm ( 7 : 43 ) . Several lesions are also noted within the left lobe . Two 12 and 9 mm respectively lesions are 
seen in segment III ( 7 : 41 ) . There is no intrahepatic biliary dilatation . Numerous dependent high - density 
foci are noted within the gallbladder consistent with gallstones . The pancreas , spleen , adrenal glands , kidneys
, and intra - abdominal loops of small and large bowel are unremarkable . There is no free air or free fluid within
the abdomen . No pathologically enlarged mesenteric or retroperitoneal lymph nodes . CT OF THE PELVIS WITH IV 
CONTRAST : The bladder , rectum , sigmoid colon , uterus and adnexa are unremarkable aside from scattered sigmoid 
diverticula . No free fluid is noted within the pelvis . No pathologically enlarged inguinal or pelvic lymph nodes 
. OSSEOUS STRUCTURES : Extensive osseous metastasis are stable compared to _ _ _ . Note is again made of multiple 
left posterior rib fractures , several of which are displaced . These fractures involve ribs 3 through 11 . 
IMPRESSION : 1 . Disease progression with interval increase in size and number of numerous , diffuse liver 
metastases compared to the CT of _ _ _ 2 . Essentially stable osseous metastatic disease . 3 . Cholelithiasis . 4 .
Left mastectomy .

In [ ]:
prompt = 'Your input fields are:\n1. `input_text` (str): radiology report text\nYour output fields are:\n1. `reasoning` (str): \n2. `extracted_entities` (list[str]): Extracted clinical entities from the report\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## input_text ## ]]\n{input_text}\n\n[[ ## reasoning ## ]]\n{reasoning}\n\n[[ ## extracted_entities ## ]]\n{extracted_entities}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Given the fields `input_text`, produce the fields `extracted_entities`.'

examples = [
    lx.data.ExampleData(
        text="STUDY : CT torso . HISTORY : Metastatic breast cancer and recently diagnosed squamous cell cancer .",
        extractions=[
            lx.data.Extraction(
                extraction_class="problem",
                extraction_text="Metastatic breast cancer",
                attributes={"assertion": "historical", "urgency": "high"}
            ),
            lx.data.Extraction(
                extraction_class="problem",
                extraction_text="squamous cell cancer",
                attributes={"assertion": "recent"}
            )
        ]
    )
]

In [19]:
result = lx.extract(
    text_or_documents=" ".join(data[1]['tokens']),
    prompt_description=prompt,
    examples=examples,
    model_id="gemini-2.5-flash",
    extraction_passes=3,    # Improves recall through multiple passes
    max_workers=20,         # Parallel processing for speed
    max_char_buffer=1000    # Smaller contexts for better accuracy
)

/Users/williamthompson/Code/projects/clinical-entity-extraction/radextract/.venv/lib/python3.13/site-packages/langextract/extraction.py:186: UserWarning: batch_length (10) < max_workers (20). Only 10 workers will be used. Set batch_length >= max_workers for optimal parallelization.
  warnings.warn(


In [20]:
cons.print(result)

AnnotatedDocument(
    extractions=[
        Extraction(
            extraction_class='problem',
            extraction_text='melanoma',
            char_interval=CharInterval(start_pos=88, end_pos=96),
            alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>,
            extraction_index=1,
            group_index=0,
            description=None,
            attributes={'assertion': 'present', 'urgency': 'high'}
        ),
        Extraction(
            extraction_class='problem',
            extraction_text='partial right and left pneumonectomy',
            char_interval=CharInterval(start_pos=687, end_pos=723),
            alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>,
            extraction_index=2,
            group_index=1,
            description=None,
            attributes={'assertion': 'status post', 'urgency': 'null'}
        ),
        Extraction(
            extraction_class='problem',
            extraction_text='soft tissue irregularity surrounding the suture line in the right lung base',
            char_interval=CharInterval(start_pos=744, end_pos=819),
            alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>,
            extraction_index=3,
            group_index=2,
            description=None,
            attributes={'assertion': 'present', 'urgency': 'low'}
        ),
        Extraction(
            extraction_class='problem',
            extraction_text='enlargement',
            char_interval=CharInterval(start_pos=840, end_pos=851),
            alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>,
            extraction_index=4,
            group_index=3,
            description=None,
            attributes={'assertion': 'no evidence of', 'urgency': 'null'}
        ),
        Extraction(
            extraction_class='problem',
            extraction_text='increased nodularity',
            char_interval=CharInterval(start_pos=855, end_pos=875),
            alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>,
            extraction_index=5,
            group_index=4,
            description=None,
            attributes={'assertion': 'no evidence of', 'urgency': 'null'}
        ),
        Extraction(
            extraction_class='problem',
            extraction_text='nodule within the periphery of right upper lobe',
            char_interval=CharInterval(start_pos=914, end_pos=961),
            alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>,
            extraction_index=1,
            group_index=0,
            description=None,
            attributes={'assertion': 'current', 'urgency': 'low'}
        ),
        Extraction(
            extraction_class='problem',
            extraction_text='larger solid nodule centrally in the right upper lobe',
            char_interval=CharInterval(start_pos=1034, end_pos=1053),
            alignment_status=<AlignmentStatus.MATCH_LESSER: 'match_lesser'>,
            extraction_index=2,
            group_index=1,
            description=None,
            attributes={'assertion': 'current', 'urgency': 'low'}
        ),
        Extraction(
            extraction_class='problem',
            extraction_text='left - sided subcutaneous superficial soft tissue enhancing nodule at the axillary 
level',
            char_interval=CharInterval(start_pos=1412, end_pos=1500),
            alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>,
            extraction_index=3,
            group_index=2,
            description=None,
            attributes={'assertion': 'current', 'urgency': 'high'}
        ),
        Extraction(
            extraction_class='problem',
            extraction_text='sebacious cyst',
            char_interval=CharInterval(start_pos=1641, end_pos=1655),
            alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>,
            extraction_index=4,
            group_index=3,
            description=None,
            attributes={'assertion': 'possible', 